In [28]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, DecimalType, DoubleType
from pyspark.sql import SparkSession

In [29]:
spark = SparkSession \
    .builder \
    .config("fs.s3a.access.key", "Q3AM3UQ867SPQQA43P2F") \
    .config("fs.s3a.secret.key", "zuf+tfteSlswRu7BJ86wekitnifILbZam1KYY3TG") \
    .config("fs.s3a.endpoint", "play.min.io:9000") \
    .appName("VolumeCalculation") \
    .getOrCreate()

In [ ]:
sc = spark.sparkContext

sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")

In [31]:
token_transfers_schema = StructType([ \
    StructField("token_address", StringType(), True), \
    StructField("from_address", StringType(), True), \
    StructField("to_address", StringType(), True), \
    StructField("value", DecimalType(38, 0), True), \
    StructField("transaction_hash", StringType(), True), \
    StructField("log_index", LongType(), True), \
    StructField("block_number", LongType(), True), \
  ])

In [33]:
tokens_schema = StructType([ \
    StructField("address", StringType(), True), \
    StructField("symbol", StringType(), True), \
    StructField("name", StringType(), True), \
    StructField("decimals", LongType(), True), \
    StructField("total_supply", LongType(), True), \
    StructField("block_number", LongType(), True), \
  ])

In [35]:
basePath = "s3a://bsc-test-tx-volume"

In [37]:
token_transfers_df = spark.read.format("csv") \
    .option("header", True) \
    .schema(token_transfers_schema) \
    .load(basePath + "/token_transfers/*/*/*.csv")

In [38]:
tokens_df = spark.read.format("csv") \
    .option("header", True) \
    .schema(tokens_schema) \
    .load(basePath + "/tokens/*.csv")

In [39]:
token_transfers_df.show(10, False)

+------------------------------------------+------------------------------------------+------------------------------------------+----------------------+------------------------------------------------------------------+---------+------------+
|token_address                             |from_address                              |to_address                                |value                 |transaction_hash                                                  |log_index|block_number|
+------------------------------------------+------------------------------------------+------------------------------------------+----------------------+------------------------------------------------------------------+---------+------------+
|0x6df52fd8732c0d149035e16f4536ef0a61487056|0xcd3d455284b8989ae83459bac9307901a9e21a09|0xe9e0828e243e5930cb478528f01c02242d8ba0fd|7000000000000000000   |0x8d9627c88d8d3b08295dcbf82d16652df69b96310d6b9c14c0157bfb21a026ba|116      |28600002    |
|0x55d398326f99059ff7754

In [40]:
tokens_df.show(10, False)

+------------------------------------------+------+-----------------+--------+---------------+------------+
|address                                   |symbol|name             |decimals|total_supply   |block_number|
+------------------------------------------+------+-----------------+--------+---------------+------------+
|0x55d398326f99059ff775485246999027b3197955|USDT  |Tether USD       |18      |null           |null        |
|0x6df52fd8732c0d149035e16f4536ef0a61487056|AI    |AI               |18      |null           |null        |
|0x2cd7ca738e568589bc1c0875c0d6dec867f41bfa|FstLP |FstSwap          |18      |614228145616886|null        |
|0x04fa9eb295266d9d4650edcb879da204887dc3da|OSK   |OSK              |18      |null           |null        |
|0x7d6801e8cf716e6a06914b7106578f3beb40e51d|W3    |W3Swap Token     |18      |null           |null        |
|0xc9882def23bc42d53895b8361d0b1edc7570bc6a|FIST  |FistToken        |6       |200000000000000|null        |
|0xfa82075a6d8f85be9146e64e0

In [41]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

In [42]:
token_transfers_df.createOrReplaceTempView("token_transfers_df")
tokens_df.createOrReplaceTempView("tokens_df")

In [52]:
tokens_result_df = spark.sql("""
SELECT DISTINCT(tt.token_address)
FROM token_transfers_df tt
LEFT JOIN tokens_df t ON LOWER(tt.token_address) = LOWER(t.address)
WHERE t.address IS NULL
""")


In [53]:
result_df.show(10, False) if result_df.count() > 0 else 0

+------------------------------------------+
|token_address                             |
+------------------------------------------+
|0xaee4164c1ee46ed0bbc34790f1a3d1fc87796668|
+------------------------------------------+



In [ ]:
# result_df.repartition(1) \
#     .write \
#     .option("header",True) \
#     .csv(basePath + "/token_transfer_addresses/")